In [13]:
import logging
import pandas as pd
import os

from openpyxl import load_workbook
from datetime import datetime

In [14]:
def extract_ridership(col_list) -> dict:
    """ Extracts the ridership values from a list of list and puts it in a dictionary.
        This function already partially cleans the data by removing values that are not integers (i.e, str and None types)

    Args:
        col_list (list): This is a list of list that contains ridership values for each station—grouped by columns.

    Returns:
        dict: This a dictionary containing the ridership numbers as its values and the station names as its keys
    """
    
    station_riders_dict = {}
    
    for col in col_list:
        
        value_list = []
        str_list = []
        
        for cell in col:
            if isinstance(cell, int):
                value_list.append(cell)
            
            if isinstance(cell, str):
                str_list.append(cell)   
        
        station_name = str_list[0]
        
        station_riders_dict[station_name] = value_list
    
    return station_riders_dict

In [15]:
def extract_hours(row_list) -> list:
    """ Extracts the hour values from a list of list and puts it in a list.
        This function only extracts the hour values that have ridership values associated with it.

    Args:
        row_list (list): This is a list of list that contains values for all stations—grouped by rows.

    Returns:
        list: This a list containing the hours that have values associated with it (i.e., non-empty entries)
    """
    
    hours_list = []
    
    for row in row_list:
        if isinstance(row[5], int):
            hours_list.append(row[0])
    
    return hours_list

In [16]:
def generate_date(row_list, excel_file) -> list:
    """Generates date associated with the entries from the table.
       The assumption of this function is that if the cell contains the string, "Entry",
       then it means that following set of values belong to a new day/date.
        
    Args:
        row_list (list): This is a list of list that contains values for all stations—grouped by rows.

    Returns:
        list: This is a list containing the dates that have values associated with it (i.e., non-empty entries)
    """
    
    dates_list = []
    day = 0
    year_month = excel_file.split(".")[0]
    
    for row in row_list:
        if row[1] == "Entry":
            day += 1
        
        if isinstance(row[5], int):
            dates_list.append(f"{year_month}-{day}")
        
    return dates_list

In [17]:
def troubleshoot(hours_list, dates_list, station_riders_dict,excel_file):
    logging.basicConfig(level=logging.INFO,
                        filename=f"{datetime.strftime(datetime.now(), '%d-%B-%Y')}.log",
                        filemode="w",
                        format="%(levelname)s - %(message)s")
    
    hours_count = len(hours_list)
    dates_count = len(dates_list)
    station_count = {}
    
    for station, values in station_riders_dict.items():
        station_count[f"{station}"] = len(values)
    
    for station, count in station_count.items():
        logging.info(f"{station} element count: {count}")
        
        
    print(f"{excel_file}")
    print(f"Hour element count: {hours_count}")
    print(f"Date element count: {dates_count}")
    print(f"Station element count: {station_count}")
    print("-----------------------------------------------------------------------------------")
    

In [20]:
#  list of all available excel files from "data" folder
excel_file_list = os.listdir("data")

hours_list = []
dates_list = []
station_dict = {}

for excel_file in excel_file_list:
    
    try:
        daily = load_workbook(filename=f"data/{excel_file}")["Daily"]
        
        daily_total_row = daily.max_row
        
        daily_col_range = daily.iter_cols(min_col=2, max_col=27, min_row=1, max_row=daily_total_row, values_only=True)
        daily_row_range = daily.iter_rows(min_row=1, max_row=daily_total_row, min_col=1, max_col=27, values_only=True)
        
        daily_cols = [col for col in daily_col_range]
        daily_rows = [row for row in daily_row_range]
        
        hours = extract_hours(daily_rows)
        dates = generate_date(daily_rows, excel_file)
        station_riders_dict = extract_ridership(daily_cols)
        
        for hour in hours:
            hours_list.append(hour)
        
        for date in dates:
            dates_list.append(date)
            
        # for station, value in extract_ridership(daily_cols).items():
        #     station_list[f"{station}"] = value
        
    except ValueError:
        pass

2019-1.xlsx
Hour element count: 630
Date element count: 630
Station element count: {'North Ave': 630, 'Exit': 630, 'Quezon Ave': 630, 'GMA Kamuning': 630, 'Cubao': 630, 'Santolan': 630, 'Ortigas': 630, 'Shaw Blvd': 630, 'Boni Ave': 630, 'Guadalupe': 630, 'Buendia': 630, 'Ayala Ave': 630, 'Magallanes': 630, 'Taft': 630}
-----------------------------------------------------------------------------------
2019-10.xlsx
Hour element count: 627
Date element count: 627
Station element count: {'North Ave': 627, 'Exit': 627, 'Quezon Ave': 627, 'GMA Kamuning': 627, 'Cubao': 627, 'Santolan': 627, 'Ortigas': 627, 'Shaw Blvd': 627, 'Boni Ave': 627, 'Guadalupe': 627, 'Buendia': 627, 'Ayala Ave': 627, 'Magallanes': 627, 'Taft': 627}
-----------------------------------------------------------------------------------
2019-11.xlsx
Hour element count: 609
Date element count: 609
Station element count: {'North Ave': 607, 'Exit': 607, 'Quezon Ave': 609, 'GMA Kamuning': 609, 'Cubao': 609, 'Santolan': 609, 'O